https://www.scirp.org/journal/paperinformation.aspx?paperid=67010

Example 1

$F_1(x_1,x_2) = e^{x_1} + x_1 x_2 - 1 = 0$

$F_2(x_1,x_2) = \sin(x_1 x_2) + x_1 + x_2 - 1 = 0$

Solution: $(x_1, x_2) = (0, 1)$

# Import libraries and modules

In [10]:
import math

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras import layers

# Set random seeds for reproducibility
seed_value = 2023
tf.random.set_seed(seed_value)

# Functions

In [27]:
def F1 (x1, x2):

    return math.exp(x1) + x1 * x2 - 1

def F2 (x1, x2):

    return math.sin(x1 * x2)  + x1 + x2 - 1


# Create a custom layer using Keras layers
class Layer2(layers.Layer):
    def __init__(self, **kwargs):
        super(Layer2, self).__init__(**kwargs)

    def call(self, inputs):
        x1_val = inputs[0, 0]
        x2_val = inputs[0, 1]
        exp_func = tf.exp(x1_val)
        x1x2 = tf.multiply(x1_val, x2_val)
        sin_func = tf.sin(x1x2)

        layer_act = tf.constant([
            [
                exp_func,
                x1x2,
                0.0,  # Change this to 0.0 to create a 2D tensor
                sin_func,
                x1_val,
                x2_val,
            ]
        ], dtype=tf.float32)

        layer_act = tf.reshape(layer_act, [1, 6])
        return layer_act


# define activation function for layer2
def activation_layer2_ex1(layer):
    x1_val = tf.squeeze(layer[0, 0])
    x2_val = tf.squeeze(layer[0, 1])
    exp_func = tf.exp(x1_val)
    x1x2 = tf.multiply(x1_val, x2_val)
    sin_func = tf.sin(x1x2)

    layer_act = [
        exp_func,
        x1x2,
        0.0,  # Change this to 0.0 to create a 2D tensor
        sin_func,
        x1_val,
        x2_val,
    ]
    
    # print(layer_act)

    # Convert layer_act to a TensorFlow tensor
    layer_2 = tf.convert_to_tensor(layer_act, dtype=tf.float32)
    
    # print(type(layer_2))
    # print(layer_2)
    
    layer_2 = tf.reshape(layer_2, [1, 6])

    return layer_2


def activation_layer2_ex3(layer):
    
    x1_val = tf.squeeze(layer[0, 0])
    x2_val = tf.squeeze(layer[0, 1])
    exp_func = tf.exp(x2_val)
    x1_3 = tf.pow(x1_val, 3)

    layer_act = [
        x1_val,
        x1_3,
        x2_val,  # Change this to 0.0 to create a 2D tensor
        x1_val,
        x2_val,
        exp_func,
    ]

    # Convert layer_act to a TensorFlow tensor
    layer_2 = tf.convert_to_tensor(layer_act, dtype=tf.float32)
    layer_2 = tf.reshape(layer_2, [1, 6])

    return layer_2



# Create model
def multilayer_perceptron(x, weights, biases):

    # Reshape input if necessary, matching the shape of the first layer's weights
    x = tf.reshape(x, [1, -1])  # Adjust the shape as needed

    layer_1 = tf.add(tf.matmul(x, weights['w12']), biases['b12'])

    # layer_2 = tf.add(tf.matmul(layer_1, tf.transpose(weights['w23'])), biases['b23'])
    layer_2 = activation_layer2_ex1(layer_1)
    # layer_2 = activation_layer2_ex3(layer_1)
    
    # Output fully connected layer
    output = tf.add(tf.matmul(layer_2, weights['w34']), biases['out'])
    
    return output, layer_1


def loss_function(weights, biases):
    
    output, _= multilayer_perceptron(tf.constant(1.0, dtype=tf.float32), weights, biases)

    return tf.reduce_mean(tf.square(output))


# Train step
def train_step(weights, biases, optimizer):

    with tf.GradientTape() as tape:
        
        loss = loss_function(weights, biases)

    trainable_variables = [weights['w12']]  # list containing only 'w12'
    
    gradients = tape.gradient(loss, trainable_variables)
    optimizer.apply_gradients(zip(gradients, trainable_variables))

    return loss        


# Constants

In [28]:
training_steps = 5001  #   5000 + 1
display_step = training_steps // 10

learning_rate = 1e-2

# Model

## Create model

In [29]:
# Network Parameters
num_input = 1 # input layer
num_hidden = [2, 6]
num_output = 2 # output layer


# Given by whether the functions in F1 and F2 contains the variables x1 and x2
w23_flags = [[True, True,  False, True, True, False], [False, True, False, True, False, True]]  
w23_flags = tf.constant(w23_flags, dtype=tf.bool)

# Initialize the weights (w23) with zeros
w23 = tf.constant(tf.zeros(num_hidden, dtype=tf.float32))
# Set the weights to 1 where func_flags is True
w23 = tf.where(w23_flags, tf.ones_like(w23), w23)
w23 = tf.transpose(w23)

# Store layers weight & bias
weights = {
    # Variables x1 and x2
    'w12': tf.Variable(tf.random.normal([num_input, num_hidden[0]])),
    # Whether the functions in F1 and F2 contain the variables x1 and x2
    'w23': w23,
    # The coefficients of the functions in F1 and F2
    'w34': tf.constant([[1, 0], [1, 0], [0, 0], [0, 1], [0, 1], [0, 1]], dtype=tf.float32),
    # 'w34': tf.constant([[3, 0], [1, 0], [1, 0], [0, 1], [0, 2], [0, 1]], dtype=tf.float32),
}

biases = {
    'b12': tf.constant([0, 0], dtype=tf.float32),
    'b23': tf.constant(tf.zeros([num_hidden[1]], dtype=tf.float32)),
    'out': tf.constant([[-1, -1]], dtype=tf.float32),
    # 'out': tf.constant([[1, -2]], dtype=tf.float32),
}

# Stochastic gradient descent optimizer.
optimizer = Adam(learning_rate=learning_rate, name='custom_optimizer_name')

In [30]:
weights['w12']

<tf.Variable 'Variable:0' shape=(1, 2) dtype=float32, numpy=array([[0.23773831, 0.67823654]], dtype=float32)>

In [31]:
x_aux = tf.Variable(tf.random.normal([num_input, num_hidden[0]])).numpy()
x_aux

array([[-1.825684 , -0.9886521]], dtype=float32)

## Train model

In [32]:
for i in range(training_steps):
       
    current_loss = train_step(weights, biases, optimizer)
    if i % display_step == 0:
        print(f"epoch {i} => loss: {current_loss:.16e} ")

[<tf.Tensor: shape=(), dtype=float32, numpy=1.2683772>, <tf.Tensor: shape=(), dtype=float32, numpy=0.16124281>, 0.0, <tf.Tensor: shape=(), dtype=float32, numpy=0.16054502>, <tf.Tensor: shape=(), dtype=float32, numpy=0.23773831>, <tf.Tensor: shape=(), dtype=float32, numpy=0.67823654>]
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([1.2683772  0.16124281 0.         0.16054502 0.23773831 0.67823654], shape=(6,), dtype=float32)
epoch 0 => loss: 9.5214329659938812e-02 
[<tf.Tensor: shape=(), dtype=float32, numpy=1.2557567>, <tf.Tensor: shape=(), dtype=float32, numpy=0.15218319>, 0.0, <tf.Tensor: shape=(), dtype=float32, numpy=0.15159646>, <tf.Tensor: shape=(), dtype=float32, numpy=0.22773841>, <tf.Tensor: shape=(), dtype=float32, numpy=0.6682368>]
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([1.2557567  0.15218319 0.         0.15159646 0.22773841 0.6682368 ], shape=(6,), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=1.243316>, <tf.Tensor: sha

[<tf.Tensor: shape=(), dtype=float32, numpy=1.0952812>, <tf.Tensor: shape=(), dtype=float32, numpy=0.06143195>, 0.0, <tf.Tensor: shape=(), dtype=float32, numpy=0.06139332>, <tf.Tensor: shape=(), dtype=float32, numpy=0.09101117>, <tf.Tensor: shape=(), dtype=float32, numpy=0.6749936>]
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([1.0952812  0.06143195 0.         0.06139332 0.09101117 0.6749936 ], shape=(6,), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=1.0911812>, <tf.Tensor: shape=(), dtype=float32, numpy=0.05970309>, 0.0, <tf.Tensor: shape=(), dtype=float32, numpy=0.05966763>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0872607>, <tf.Tensor: shape=(), dtype=float32, numpy=0.6841922>]
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor([1.0911812  0.05970309 0.         0.05966763 0.0872607  0.6841922 ], shape=(6,), dtype=float32)
[<tf.Tensor: shape=(), dtype=float32, numpy=1.0875108>, <tf.Tensor: shape=(), dtype=float32, numpy=0.058188654>, 

In [33]:
x1_val, x2_val = weights['w12'][0][0], weights['w12'][0][1]

display(
    (x1_val, x2_val),
    F1(x1_val, x2_val),
    F2(x1_val, x2_val),
)


(<tf.Tensor: shape=(), dtype=float32, numpy=1.0132758e-08>,
 <tf.Tensor: shape=(), dtype=float32, numpy=1.0>)

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

<tf.Tensor: shape=(), dtype=float32, numpy=0.0>

In [34]:
F1F2, solution  = multilayer_perceptron(tf.constant(1.0, dtype=tf.float32), weights, biases)

print(f'x1 = {solution[0][0]}, x2 = {solution[0][1]}')
print(f'Residuals: F1 = {F1F2[0][0]}, F2 = {F1F2[0][1]}')

[<tf.Tensor: shape=(), dtype=float32, numpy=1.0>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0132758e-08>, 0.0, <tf.Tensor: shape=(), dtype=float32, numpy=1.0132758e-08>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0132758e-08>, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>]
<class 'tensorflow.python.framework.ops.EagerTensor'>
tf.Tensor(
[1.0000000e+00 1.0132758e-08 0.0000000e+00 1.0132758e-08 1.0132758e-08
 1.0000000e+00], shape=(6,), dtype=float32)
x1 = 1.0132757743974707e-08, x2 = 1.0
Residuals: F1 = 0.0, F2 = 0.0
